## Pięć zasad efektywnej inżynierii promptów
1. Jasne instrukcje poprawiają trafność.
2. Przykłady stabilizują odpowiedź.
3. Zdefiniowany format odpowiedzi = przewidywalny wynik.
4. Dzielenie na kroki = lepsze i pełniejsze rozwiązania.
5. Testy i weryfikacja = bezpieczeństwo i poprawność.

In [1]:
# Import bibliotek
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

# odel bazowy
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

### Zasada 1 — Jasne instrukcje

In [2]:
# Zły prompt — niejasny, bez roli i oczekiwań
bad_prompt = "Napisz funkcję w Pythonie."
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content)

# Dobry prompt — jasno określona rola i oczekiwania
good_prompt = """Jesteś ekspertem programującym w Pythonie.
Napisz funkcję w Pythonie, która przyjmuje listę liczb całkowitych
i zwraca nową listę zawierającą tylko liczby parzyste.
Dodaj test jednostkowy w pytest."""
print("\n=== Dobry prompt ===")
print(llm.invoke(good_prompt).content)

=== Zły prompt ===
Oczywiście! Jaką funkcję chciałbyś, abym napisał? Możesz podać szczegóły dotyczące tego, co funkcja powinna robić, a ja postaram się pomóc.

=== Dobry prompt ===
Oczywiście! Poniżej znajduje się funkcja w Pythonie, która przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste. Dodatkowo umieściłem przykładowy test jednostkowy z użyciem biblioteki `pytest`.

### Funkcja filtrująca liczby parzyste

```python
def filter_even_numbers(numbers):
    """
    Funkcja przyjmuje listę liczb całkowitych i zwraca nową listę zawierającą tylko liczby parzyste.

    :param numbers: List[int] - lista liczb całkowitych
    :return: List[int] - lista liczb parzystych
    """
    return [num for num in numbers if num % 2 == 0]
```

### Test jednostkowy przy użyciu pytest

```python
import pytest

def test_filter_even_numbers():
    assert filter_even_numbers([1, 2, 3, 4, 5]) == [2, 4]
    assert filter_even_numbers([10, 15, 20, 25, 30]) == [10, 20, 30]
 

### Zasada 2 — Używaj przykładów

In [3]:
# Zadanie: wygeneruj tagi na podstawie treści strony firmy

# Zero-shot
zero_shot = """Podaj tagi opisujące firmę na podstawie tekstu strony:
Firma Lego produkuje zabawki dla dzieci."""
print("=== Zero-shot ===")
print(llm.invoke(zero_shot).content)

# One-shot
one_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.
Przykład:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Teraz:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi:"""
print("\n=== One-shot ===")
print(llm.invoke(one_shot).content)

# Few-shot
few_shot = """Podaj maksymalnie trzy tagi opisujące firmę na podstawie tekstu strony.

Przykład 1:
Tekst: Firma Lego produkuje klocki dla dzieci.
Tagi: zabawki, klocki, dzieci

Przykład 2:
Tekst: Firma Nike produkuje odzież i buty sportowe.
Tagi: sport, odzież, obuwie

Teraz:
Tekst: Firma Tesla produkuje samochody elektryczne i magazyny energii.
Tagi:"""
print("\n=== Few-shot ===")
print(llm.invoke(few_shot).content)


=== Zero-shot ===
Oto propozycje tagów opisujących firmę Lego na podstawie podanego tekstu:

- Lego
- zabawki
- dzieci
- produkcja
- kreatywność
- edukacja
- rozrywka
- budowanie
- rozwój
- marka
- innowacja
- zabawa
- klocki
- rodzinne
- design

Możesz dostosować lub dodać inne tagi w zależności od kontekstu i specyfiki firmy.

=== One-shot ===
odzież, buty, sport

=== Few-shot ===
samochody, elektryczne, energia


In [11]:
prompt = """Stwórz listę 5 sloganów reklamowych dla kawiarni.
Zastosuj styl z przykładowych sloganów załączonych poniżej:
"U nas dostaniesz kawę na ławę!"
"Kawa szybko tanio i konkretnie!"
"""
print(llm.invoke(prompt).content)

- Szybko, tanio i na poziomie – kawa, co robi robotę!
- Kawa z kopyta: aromat, energia, konkret!
- Mała czarna – wielka sprawa: od rana stawia na nogi!
- Kawa do rzeczy – bez ściemy, bez zwłoki, bez nudy!
- Wpadnij na łyk – wyjdź z powerem!


In [13]:
prompt = """Stwórz listę 5 sloganów reklamowych dla kawiarni.
Zastosuj styl z przykładowych sloganów załączonych poniżej:
"Nasza kawa jest niczym spacer w rajskim ogrodzie w słoneczny radosny dzień."
"Pijąc naszą kawę poczujesz się tak, jakby anioł zatańczył Tobie na języku."
"""
print(llm.invoke(prompt).content)

- Nasza kawa otula jak ciepły promień słońca o poranku, budząc w sercu taniec radości.
- Łyk naszej kawy to podróż na szczyt chmur, gdzie cisza smakuje czystym szczęściem.
- Pijąc naszą kawę, odkryjesz w sobie wiosnę: kwitnące myśli, śmiech i lekkość kroków.
- Nasza kawa szepcze do zmysłów jak łagodny wiatr wśród lawendowych pól.
- Zatrzymaj się przy naszej kawie, a czas rozpuści się jak karmel w promieniach popołudnia.


### Zasada 3 — Zdefiniuj format odpowiedzi
walidacja wejścia i wyjścia z wykorzystaniem biblioteki Pydantic

In [4]:
import json
json.loads("""{
	"name": "Zgadnij Kto To?",
	"description": "Gra polegająca na odgadywaniu postaci na podstawie zadawanych pytań. Dzieci uczą się zadawania pytań i logicznego myślenia.",
	"min_number_of_players": 2,
	"max_number_of_players": 6,
	"rules": "Każdy gracz wybiera jedną postać z zestawu. Pozostali gracze zadają pytania, na które można odpowiedzieć 'tak' lub 'nie', aby odgadnąć, kto to jest. Wygrywa ten, kto jako pierwszy odgadnie postać."
}""")

{'name': 'Zgadnij Kto To?',
 'description': 'Gra polegająca na odgadywaniu postaci na podstawie zadawanych pytań. Dzieci uczą się zadawania pytań i logicznego myślenia.',
 'min_number_of_players': 2,
 'max_number_of_players': 6,
 'rules': "Każdy gracz wybiera jedną postać z zestawu. Pozostali gracze zadają pytania, na które można odpowiedzieć 'tak' lub 'nie', aby odgadnąć, kto to jest. Wygrywa ten, kto jako pierwszy odgadnie postać."}

In [22]:
from pydantic import BaseModel, Field
from typing import List
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
import dotenv

dotenv.load_dotenv()

# Definicja schematu wyjścia (Pydantic)
class CityGuide(BaseModel):
    city: str = Field(..., description="Miasto, którego dotyczy przewodnik")
    summary: str = Field(..., description="Krótki opis miasta (2–3 zdania)")
    must_do: List[str] = Field(..., description="Lista 3–5 rzeczy do zrobienia")

# LLM + structured output
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(CityGuide)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od podróży. Odpowiadaj po polsku, zwięźle."),
    ("user", "Stwórz krótki przewodnik po {city} dla {days}-dniowej wizyty.")
])

chain = prompt | structured_llm

# Walidacja wejścia Pydantic + wywołanie łańcucha
from pydantic import BaseModel, Field

class GuideRequest(BaseModel):
    city: str = Field(min_length=2)
    days: int = Field(ge=1, le=7)

req = GuideRequest(city="Wrocław", days=2)

result: CityGuide = chain.invoke(req.model_dump())  # <- dostajesz OBIEKT Pydantic
print(result)                         # CityGuide(city=..., summary=..., must_do=[...])
print(result.model_dump_json(indent=2))  # JSON gotowy do zapisu/transportu

city='Wrocław' summary='Wrocław to malownicze miasto w Polsce, znane z pięknej architektury, licznych mostów i bogatej historii. Jest to miejsce, gdzie tradycja spotyka nowoczesność, a tętniące życiem rynki i urokliwe uliczki przyciągają turystów z całego świata.' must_do=['Odwiedź Rynek i zobacz Ratusz oraz kolorowe kamienice.', 'Przejdź się po Ostrowie Tumskim i zwiedź Katedrę św. Jana Chrzciciela.', 'Zobacz Panoramę Racławicką, monumentalne dzieło sztuki.', 'Spędź czas w Ogrodzie Japońskim i Parku Szczytnickim.', 'Zrób zdjęcie z wrocławskimi krasnalami, które są rozsiane po całym mieście.']
{
  "city": "Wrocław",
  "summary": "Wrocław to malownicze miasto w Polsce, znane z pięknej architektury, licznych mostów i bogatej historii. Jest to miejsce, gdzie tradycja spotyka nowoczesność, a tętniące życiem rynki i urokliwe uliczki przyciągają turystów z całego świata.",
  "must_do": [
    "Odwiedź Rynek i zobacz Ratusz oraz kolorowe kamienice.",
    "Przejdź się po Ostrowie Tumskim i zwie

### Zasada 4 — Dziel złożone zadania na kroki

#### Przygotowanie planu wycieczki

In [23]:
# Zły prompt — wszystko naraz
bad_prompt = """Przygotuj trzydniowy plan zwiedzania Poznania z budżetem 300 euro,
uwzględniając atrakcje, restauracje, transport i mapy."""
print("=== Zły prompt ===")
print(llm.invoke(bad_prompt).content[:600], "...")

# Dobry prompt — krok po kroku
good_step1 = "Wypisz najważniejsze atrakcje kulturalne w Poznaniu z godzinami otwarcia."
step1 = llm.invoke(good_step1).content
print("\n=== Dobry prompt — krok 1 ===")
print(step1[:600], "...")

good_step2 = f"Na podstawie tej listy ułóż plan zwiedzania na 3 dni, maks 4 atrakcje dziennie. Atrakcje: {step1}"
step2 = llm.invoke(good_step2).content
print("\n=== Dobry prompt — krok 2 ===")
print(step2[:600], "...")


=== Zły prompt ===
Oto trzydniowy plan zwiedzania Poznania z budżetem 300 euro. Plan uwzględnia atrakcje turystyczne, restauracje, transport oraz mapy.

### Dzień 1: Stare Miasto i okolice

**Rano:**
- **Śniadanie:** Kawiarnia "Café La Ruina" (około 5 euro)
- **Atrakcja:** Stary Rynek – zwiedzanie Ratusza, zobaczenie koziołków (bezpłatnie)
- **Transport:** Spacer po Starym Mieście (bezpłatnie)

**Południe:**
- **Obiad:** Restauracja "Bamberka" – tradycyjne dania wielkopolskie (około 10 euro)
- **Atrakcja:** Muzeum Narodowe w Poznaniu (bilet wstępu około 5 euro)

**Popołudnie:**
- **Atrakcja:** Ostrów Tumski – Ka ...

=== Dobry prompt — krok 1 ===
Oto lista najważniejszych atrakcji kulturalnych w Poznaniu wraz z ich godzinami otwarcia (godziny mogą się różnić w zależności od sezonu, dni tygodnia i świąt, dlatego zawsze warto sprawdzić aktualne informacje na oficjalnych stronach):

1. **Stary Rynek**
   - Opis: Serce Poznania, z pięknymi kamienicami, ratuszem i fontanną Prozerpiny.
   - 

#### Generowanie artykułu w krokach

In [17]:
ARTICLE_SUBJECT = "Jakie kompetencje będą najcenniejsze u programistów za 10 lat."

plan = f"Przygotuj plan składający sie z 3 punktów artykułu na temat: {ARTICLE_SUBJECT}."
planGenerated = llm.invoke(plan).content
print(planGenerated, "...")

1) AI‑native development i współpraca z modelami
- Projektowanie systemów z LLM/agentami jako komponentami (kontekst, pamięć, tool-use, API).
- LLMOps/MLOps: wersjonowanie promptów/modeli, ewaluacje, monitoring, bezpieczeństwo i zgodność.
- Inżynieria jakości: testy generatywne, property-based, statyczna analiza, weryfikacja formalna w krytycznych obszarach; data literacy (RAG, prywatność danych).

2) Mistrzostwo systemowe i fundamenty inżynierskie
- Głębokie podstawy CS: algorytmy, współbieżność, systemy rozproszone, architektury odporne i zdarzeniowe.
- Bezpieczeństwo i prywatność by design: kryptografia praktyczna, threat modeling, supply‑chain security.
- Wydajność i efektywność: FinOps/GreenOps, profilowanie, praca z heterogenicznym sprzętem (GPU/TPU/edge, serverless).

3) Kompetencje produktowe i międzyludzkie
- Myślenie produktowe: discovery, praca z danymi i metrykami, eksperymentowanie, UX‑empatia.
- Komunikacja i współpraca międzydyscyplinarna, leadership w zespołach wspieran

In [27]:
MAX_WORDS_IN_PARAGRAPH = 200

prevParagraphContent = ""

for i in range(1,4):
    paragraphPrompt = f"Przygotuj tylko treść {i}. akapitu artykułu: {ARTICLE_SUBJECT}. Maksymalna długość tekstu to {MAX_WORDS_IN_PARAGRAPH}. Wygeneruj tylko tekst jednego paragrafu. Plan całego artykułu:\n {planGenerated}. Treść poprzednich akapitów: {prevParagraphContent}."
    paragraphContent = llm.invoke(paragraphPrompt).content
    prevParagraphContent += paragraphContent + "\n"
    print(f"Akapit {i}.", paragraphContent, "\n\n")

Akapit 1. W ciągu najbliższych dziesięciu lat kompetencje programistów będą musiały ewoluować w odpowiedzi na dynamiczny rozwój technologii, zwłaszcza w obszarze sztucznej inteligencji. Kluczowym elementem stanie się AI-native development, który zakłada projektowanie systemów z wykorzystaniem modeli językowych (LLM) oraz agentów jako integralnych komponentów. Programiści będą musieli opanować umiejętności związane z LLMOps i MLOps, obejmujące wersjonowanie promptów, monitorowanie oraz zapewnienie bezpieczeństwa i zgodności. W kontekście rosnącej złożoności systemów, inżynieria jakości zyska na znaczeniu, a umiejętność przeprowadzania testów generatywnych oraz analizy danych stanie się niezbędna, aby sprostać wymaganiom nowoczesnych aplikacji. W obliczu tych zmian, programiści będą musieli nie tylko rozwijać swoje umiejętności techniczne, ale także doskonalić kompetencje produktowe i międzyludzkie, aby skutecznie współpracować w złożonych, interdyscyplinarnych zespołach. 


Akapit 2. Mi

### Zasada 5 — Testuj i weryfikuj wyniki

In [7]:
from langchain.evaluation import load_evaluator
from dotenv import load_dotenv

load_dotenv()

evaluator = load_evaluator("embedding_distance", embeddings_model="openai")

result = evaluator.evaluate_strings(
    prediction="Stolica Polski to Warszawa",
    reference="Stolica Warszawa?"
)

print(result)


{'score': 0.07116926852407579}
